%md
# Analisis de Sentimientos 
#####  Procesamiento del Lenguaje Natural - NLP con Transformers(IA)<br>
Utilización de Modelos del lenguaje grande para la clasificación de sentimientos y librerías especializadas para trabajar comentarios y opinones de films 


<img src="https://upload.wikimedia.org/wikipedia/commons/6/63/Databricks_Logo.png" width="145">
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS0l3LtGUQZES0xqGbA-9ATmpO6yMe6GrIHuQ&s" width="130">
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ-NEICv1aGTvDRncdvM_fXoah5SNWx4pXAvg&s" width="110">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Scikit_learn_logo_small.svg/3840px-Scikit_learn_logo_small.svg.png" width="115">

Este proyecto explora cómo los modelos de Procesamiento de Lenguaje Natural (NLP) pueden identificar y clasificar las emociones presentes en reseñas de películas.   

A partir de más de 10,000 comentarios de usuarios, se aplican técnicas de limpieza, tokenización y lematización para descubrir patrones de opinión y tendencias según género, calificación y palabras más frecuentes.

**Sobre el Dataset**    
Este dataset (corpus) contiene criticas realizadas por los usuarios de www.filmaffinity.com sobre todas las películas y series españolas (Mas de 10000 peliculas).   


El dataset (copus) esta formado por:    

- film_name: Título de la película.     
- gender: Genero de la película (comedia, terror, acción, etc.)         
- film_avg_rate: Nota media de la película (votos de todos los usuarios)      
- review_rate: Nota que el usuario que hace la crítica pone a la película.      
- review_title: Título de la crítica.     
- review_text: Crítica de la película   


Este conjunto de datos es la base para aplicar técnicas de Procesamiento de Lenguaje Natural (NLP), tokenización y lematización, con el objetivo de descubrir las palabras más frecuentes, las emociones predominantes y las diferencias según género o calificación.

#### Instalaciones de librerias y dependencias

In [0]:
#Se instalan las siguiente librerias y paquetes para procesamiento en el entorno de Databricks
%pip install nltk
%pip install spacy
%pip install --upgrade typing_extensions
%pip install plotnine
%pip install wordcloud
%pip install transformers torch
!python -m spacy download es_core_news_sm

#%restart_python


In [0]:
import pandas as pd
import numpy as np

reviews = pd.read_csv('film_reviews_result.csv', sep= "|", )
reviews.sample(5)
#examinacion de valores nulo

reviews.info()

Se realizó un análisis para identificar posibles valores faltantes en el dataset.
El resultado muestra que no existen valores nulos en ninguna de las columnas, lo que garantiza que los datos están completos y listos para ser procesados sin necesidad de imputaciones o correcciones adicionales

#### Funcion de prosesamiento de datos y refinamiento de diccionarios


Se construyó una función cuyo objetivo es realizar la **limpieza y preparación de los datos** para el análisis descriptivo de los comentarios de películas.

El proceso incluye:

- Instalación y uso de las bibliotecas necesarias para el tratamiento de texto.

- Transformación del dataset en un objeto pandas para un manejo interactivo.

- Eliminación de números y caracteres irrelevantes, normalización de espacios y texto.

- **Tokenización y lematización** de los comentarios.

Con este flujo se obtiene un corpus estandarizado que permite identificar las **palabras más frecuentes** en las reseñas, tanto en el conjunto completo como segmentado por género de película. Esto constituye la base para el análisis de sentimientos y la exploración de patrones en las opiniones de los usuarios.

In [0]:
import re, string
from nltk.corpus import stopwords
import spacy
import nltk
nltk.download('stopwords') #Descargamos stop works en el caso de que no este descargado

nlp_lematizar = spacy.load('es_core_news_sm') #reducir palabras a su forma base o lema (ej. “corriendo” → “correr”).

palabras_vacias = list(stopwords.words('spanish'))

#Agreganos mas palabras que no tienen alguna carga emocional a las palabras vacias de la libreria
#Refinado del diccionarios
filtros_cine_pro = [
    # Tu lista original + correcciones
    'peliculas', 'películas', 'film', 'filme', 'films', 'cine', 'pelcula', 'ser', 'haber', 'tener', 'ms', 
    'pantalla', 'pantallas', 'obra', 'obras', 'produccion', 'producción', 'estreno', 'est',
    'director', 'directores', 'actor', 'actriz', 'actores', 'actrices', 'reparto', 'ver', 'bien',
    'personaje', 'personajes', 'protagonista', 'protagonistas', 'guion', 'guión', 
    'guionista', 'papel', 'papeles', 'elenco', 'trama', 'historia', 'historias',  
    'escena', 'escenas', 'final', 'inicio', 'principio', 'desenlace', 'momento', 
    'momentos', 'secuencia', 'secuencias', 'parte', 'partes', 'capitulo', 'capítulo', 
    'argumento', 'visto', 'viendo', 'mirar', 'hacer', 'hace', 'hecho', 'hacia', 'si', 
    'parece', 'parecen', 'parecer', 'creo', 'creer', 'pienso', 'pensar', 'decir', 
    'dice', 'dicho', 'contar', 'cuenta', 'pasa', 'pasar', 'pasado', 'va', 'ir', 'ido', 
    'tiene', 'tienen', 'querer', 'quiere', 'deja', 'dejar', 'dar', 'da', 'solo', 
    'sólo', 'tan', 'muy', 'mas', 'pero', 'aunque', 'entonces', 'luego', 'despues', 
    'después', 'antes', 'siempre', 'nunca', 'jamas', 'jamás', 'asi', 'así', 'donde', 
    'cuando', 'quien', 'quienes', 'cual', 'cuales', 'todo', 'toda', 'todos', 'todas', 
    'algo', 'nada', 'algun', 'algún', 'alguna', 'algunos', 'algunas', 'otro', 'otra', 
    'otros', 'otras', 'cada', 'ambos', 'mismo', 'misma', 'mismos', 'mismas', 
    'bastante', 'demasiado', 'mucho', 'mucha', 'muchos', 'muchas', 'varios', 'varias',
    'total', 'realmente', 'verdad', 'verdaderamente', 'simplemente', 'posible', 
    'veces', 'caso', 'puesto', 'punto', 'manera', 'forma', 'general', 'ejemplo',
    'mejor', 'buen', 'bueno', 'poder', 'vez', 'tambin', 'también', 
    'primero', 'primera', 'espectador', 'novela', 'libro', 's',

    # Agregadas por género extraídas de tus capturas
    'netflix', 'bayona', 'anderson', 'robert', 'rodriguez', 'momoa', 'wes', 'anna', 'dahl', 
    'alguno', 'jennifer', 'lawrence', 'adam', 'dicaprio', 'spy', 'kids', 'lindsay', "pelcular"
    'serie', 'temporada', 'episodio', 'videojuego', 'animacin', 'genero', 'gnero', 
    'clich', 'cliche', 'original', 'franquicia', 'saga', 'cosa', 'cosas', 'mundo', 
    'vida', 'tiempo', 'llegar', 'quedar', 'gran', 'menos', 'dos', 'tres', 'alguno', 
    'as', 'lugar', 'lado', 'cierto', 'claro', 'pues', 'sino', 'tampoco', 'adems', 
    'además', 'cmo', 'como', 'aquel', 'aquello', 'aqu', 'aquí', 'mientras', 'tras', 
    'estn', 'están', 'saber', 'gustar', 'vivir', 'seguir', 'encontrar', 'terminar',
    'volver', 'poner', 'resultar', 'cinta', 'trabajo', 'tema', 'sociedad', 'realidad', 
    'social', 'politico', 'pblico', 'público', 'humano', 'minuto', 'hora', 'pasado', 
    'nuevo', 'nueva', "pelcular", "nio", "que", "ao"
]
palabras_vacias.extend(filtros_cine_pro)
print(palabras_vacias[1:10])

#Cosntruccion de funcion para trabajar cada comentario
coment = "Un ejemplo de !!!película con muy buenas críticas.,/*-😊😊8😊"

def limpieza(comentario, i = " "):
  texto = comentario.lower()
  texto = texto.encode('ascii', 'ignore').decode('ascii') #eliminamos caracteristicas raras emojis etc
  texto =  re.sub(f"[{re.escape(string.punctuation)}]", "", texto) #Eliminacion de signos de puntuacion
  texto =  re.sub(r"\d+", "", texto) #Eliminar nuemeros
  texto =  re.sub(r"\s+", " ", texto).strip() #normalizar espacios
  doc = nlp_lematizar(texto)  # tokeniza y lematiza  salida objeto Doc
  tokens = [i.lemma_ for i in doc if i.lemma_ not in palabras_vacias]
  return tokens

limpieza(coment)

# Se utilizará Este objeto con el cual Se eliminará aquellas palabras que no tienen un significado Representativo dentro del comentario

In [0]:
#Este codigo debe escalarce a pyspark para preocesameinto distribuido
reviews2 = (
  reviews
  .assign(
    review_rate = lambda x: x['review_rate'].astype('float'),
    film_avg_rate = lambda x: x['film_avg_rate'].str.replace(",", ".").astype('float'),
    genero =  reviews['gender'].str.split(",").str[0], #De cada lista dentro de cada fila, dame el elemento en la posición 0
    tokenizacion =  lambda x :  x['review_text'].map(limpieza)
  )
)



In [0]:
reviews2 = (
  reviews2
  .assign(
    conteo_palabras = lambda x : x['tokenizacion'].str.len(),
  )
)

reviews2.sample(5)

In [0]:
#cuanta peliculas y comentarios se disponen

peliculas_comentarios = (
  reviews2
  .groupby('film_name', as_index=False)
  .size()
  .sort_values('size', ascending=False)
)
print("#Distribucion del numero de comentarios por pelicula",
      "\n"*2,
      peliculas_comentarios.describe(),
      "\n"*2,
      "TOP 14 - Films con mayor comentarios en el Data Set"
    )

peliculas_comentarios.reset_index(drop=True)[0:16]
#Aproximadamente exisyte 766 pelicual registradas
#vemo cuantos comentarios disponemos

Utilzaremos otro paque de python plotline ya que es la representacipon de ggplot de r en python ¡Es una excelente noticia! Para muchos analistas que vienen de R, descubrir plotnine es como volver a casa.

In [0]:
from plotnine import ggplot, aes, geom_histogram, labs, theme_light

# Creamos el gráfico
grafico_longitud = (
    ggplot(reviews2) 
    + aes(x='conteo_palabras') 
    + geom_histogram(bins=50, fill="#69b3a2", color="white")
    + labs(
        title="Distribución del número de palabras",
        subtitle="Análisis sobre 10,058 comentarios de data cruda",
        x="Cantidad de Palabras",
        y="Frecuencia"
    )
    + theme_light()
)
# Para mostrarlo
print("\n")
grafico_longitud

In [0]:
from plotnine import ggplot, aes, geom_col, labs, theme_light, coord_flip

#Gráfico de Calificación Promedio de la Película
p1 = (
    ggplot(reviews2) 
    + aes(x='film_avg_rate') 
    + geom_histogram(bins=15, fill="#392aa8", color="white")
    + labs(
        title="Puntuación General de Filmaffinity",
        subtitle="Promedio histórico calculado por la plataforma",
        x="Calificación (Escala 1-10)",
        y="Frecuencia de Películas"
    )
    + theme_light()
)

# Gráfico de Puntuación del Usuario
p2 = (
    ggplot(reviews2) 
    + aes(x='review_rate') 
    + geom_histogram(bins=10, fill="#7C7C93", color="white")
    + labs(
        title="Puntuación de las Reseñas",
        subtitle="Notas individuales otorgadas por los usuarios",
        x="Rating del Usuario (Escala 1-10)",
        y="Frecuencia de Votos"
    )
    + theme_light()
)

# 3. Unir los gráficos lado a lado sin usar los "ax" de Matplotlib (Evita el TypeError)
p1.show(), p2.show()

In [0]:
from plotnine import ggplot, aes, geom_col, labs, theme_light, coord_flip
generos = (
  reviews2
  .groupby("genero", as_index=False)
  .agg(
     {
         "film_name": "count"
     }
  )
  .sort_values("film_name", ascending=False)
  .assign(
      participacion_porce = lambda x : np.round((x["film_name"] / x["film_name"].sum())*100,2)
  )
)

p1 = (
    ggplot(generos)  # plotnine trabaja mejor con pandas
    + aes(x='reorder(genero, film_name)', y='film_name') #ordenamos el eje
    + geom_col(fill="#392aa8", color="white")
    + labs(
        title="Numero de videos por genero",
        x="Genero",
        y="Numero de Peliculas"
    )
    + theme_light()
    + coord_flip()  # opcional: mejora la lectura
)

p1.show()
generos

Dado que tenemos 10000 comentarios El proceso para realizar la obtención del sentimiento se lo hará a través de un modelo de lenguaje largo llms Utilizaremos los modelos que vienen por defecto en databriks Para hacer el procesamiento

Recordemos que nuestros Datos están almacenados en un objeto de pandas y si lo hacemos directamente utilizando El modelo de lenguaje grande Y el objeto Donde se almacena los comentarios pues tardará demasiado tiempo para tanto posito pues miraremos a Spark Para utilizar las funciones  De computación distribuida A través de Pyspark

### Analisis de palabras -  Frecuencia por Genero

 Transformación objeto dataframe Pandas a un objeto dataframe de PySpark

In [0]:
import pyspark.sql.functions as f

df_spark = spark.createDataFrame(reviews2)

df_tokens = (
    df_spark
    .select(
        f.col("genero"),
        f.explode("tokenizacion").alias("palabra")
    )
)
df_tokens.count()
df_tokens.limit(10).display()


In [0]:
df_wc = (
    df_tokens
    .groupBy(f.col('genero'), f.col('palabra'))
    .agg(
       f.count( f.col('palabra')).alias("frecuencia")
    )
    .orderBy(f.desc("frecuencia"))
)
 
df_wc.count()
df_wc.limit(50).display()

In [0]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

df_wc_pd = df_wc.toPandas()
for genero in df_wc_pd["genero"].unique()[2:]:
    subset = df_wc_pd[df_wc_pd["genero"] == genero]

    frecuencias = dict( #convierte palabras en un diccionario { "bueno": 12, "malo": 5,"excelente": 7}
        zip(subset["palabra"],
         subset["frecuencia"])
    )

    #Nube de palabras graficas
    wc = WordCloud(
        width =500,
        height=180,
        background_color="white",
        max_words= 100,
        collocations=False
    ).generate_from_frequencies(frecuencias)

    plt.figure(figsize=(10,5))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"WordCloud Genero del film: {genero}")
    plt.show()
    print("\n")

#### Inferencia con Transformer Modelos de Lenguaje

Dado que el dataset contiene más de 10,000 comentarios, se diseñó un ejercicio inicial empleando un cerebro pre entrenado sobre una muestra de **150 reseñas.** El objetivo de esta prueba es evaluar la capacidad del modelo para **extraer y clasificar sentimientos** en textos de usuarios, validando su desempeño antes de escalar el análisis completo.

In [0]:
from transformers import pipeline

#cerebro
# Modelo multilenguaje español
analizador = pipeline(
    "sentiment-analysis", 
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student"
    )

def clasificar_sentimiento(texto):
    resultado = analizador(texto[:511]) 
    label = resultado[0]['label']
    mapping = {"POS": "Positivo", "NEG": "Negativo", "NEU": "Neutral"}
    return mapping.get(label, label)


texto = "Me encantó la película, fue muy mala."
resultado = clasificar_sentimiento(texto[:512])
print(resultado)

In [0]:
# Obtencion de una muestar aleatoria de comentarios para el modelo llms

muestra_datos = reviews2.sample(500)

In [0]:
# Aplica la función en pandas
muestra_datos['sentimiento_ia'] = muestra_datos['review_text'].apply(clasificar_sentimiento)

muestra_datos.sample(1)

#### Grafica de sentimiento Positivo, Negativo y Neutral

In [0]:
from plotnine import ggplot, aes, geom_col, labs, theme_light, scale_fill_manual, geom_text

# Agrupamos los datos para tener el total general por sentimiento
df_general = (
    muestra_datos
    .groupby(['sentimiento_ia'], as_index=False)[['film_name']]
    .count()
    .assign(
        participacion = lambda x : np.round((x['film_name'] / sum(x['film_name']))*100,2)
    )
)

df_general.display()


p_sentimiento_general = (
    ggplot(df_general) 
    + aes(x='sentimiento_ia', y='film_name', fill='sentimiento_ia')
    + geom_col(show_legend=False)
    + geom_text(
        aes(label='film_name'),
        va='bottom',        # Alineación vertical (bottom para que suba)
        size=10,            # Tamaño de la letra
        format_string='{}'  # Formato del número
    )
    + scale_fill_manual(values={
        "positive": "#2ecc71", 
        "negative": "#e74c3c", 
        "neutral": "#95a5a6"
    })
    + labs(
        title="Distribución General de Sentimientos",
        subtitle="Basado en el conteo total",
        x="Sentimiento",
        y="Total de Menciones / Palabras"
    )
    + theme_light()
)

p_sentimiento_general.show()

La audiencia está totalmente polarizada con un empate técnico entre críticas negativas (49.2%) y positivas (48.6%), dejando casi sin espacio a la indiferencia o neutralidad (2.2%). Esta distribución de 246 menciones negativas contra 243 positivas refleja que la película genera una reacción emocional intensa y dividida. El impacto mediático es tan equilibrado que apenas tres comentarios inclinan la balanza, confirmando que no existe un consenso general sobre la obra.

In [0]:
from plotnine import ggplot, aes, geom_col, labs, theme_light, coord_flip

sentimiento_genero = (
    muestra_datos
    .groupby(["genero", "sentimiento_ia"], as_index=False)
    .agg({
        "film_name": "count",
        "conteo_palabras": "mean"
    })
    .sort_values("genero", ascending=False)
    .assign(
        total_por_genero = lambda x: x.groupby("genero")["film_name"].transform("sum"),
        proporcion = lambda  x : np.round(x['film_name'] /  x['total_por_genero'] * 100, 2)
    )
)

from plotnine import ggplot, aes, geom_col, labs, theme_light, scale_fill_manual, geom_text, facet_wrap, theme, element_text

p_sentimiento_general = (
    ggplot(sentimiento_genero) 
    + aes(x='sentimiento_ia', y='film_name', fill='sentimiento_ia')
    + geom_col()
    + geom_text(
        aes(label='film_name'),
        va='center',       
        nudge_y=2,
        size=10,            
        format_string='{}'  # Formato del número
    )
    + facet_wrap('~genero', scales='free_y')
    + labs(
        title="Distribución Sentimientos por Genero",
        subtitle="Basado en el conteo total de peliculas",
        x="Sentimiento",
        y="Total de Menciones / Palabras"
    )
    + theme_light()
    + theme(
        legend_position='bottom',     
        legend_direction='horizontal', 
        figure_size=(12, 10),    # Hancho 12 pulgadas, Alto 8 pulgadas
        strip_text=element_text(size=12, weight='bold'), # títulos de los géneros
        subplots_adjust={'wspace': 0.25, 'hspace': 0.35} # espaciados o pending
    )
)
p_sentimiento_general.show()


sentimiento_genero